In [95]:
from collections import namedtuple
from heapq import heappop, heappush
from random import choice
import numpy as np
import time
import functools

In [96]:
PUZZLE_SIZES = [3, 4, 5]
action = namedtuple('Action', ['pos1', 'pos2'])

In [97]:
def counter(fn):
    """Simple decorator for counting number of calls"""
    @functools.wraps(fn)
    def helper(*args, **kargs):
        result = fn(*args, **kargs)
        helper.calls += len(result)
        return result

    helper.calls = 0
    return helper

@counter
def available_actions(state: np.ndarray) -> list['Action']:
    size = state.shape[0]
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < size - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < size - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_goal(state: np.ndarray, goal: np.ndarray) -> bool:
    return np.array_equal(state, goal)


In [98]:
RANDOMIZE_STEPS = 100_000

def generate_random(size: int) -> np.ndarray:
    state = np.array([i for i in range(1, size**2)] + [0]).reshape((size, size))
    for r in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))
    return state


In [99]:

def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    size = state.shape[0]
    goal_positions = {val: (i // size, i % size) for i, val in enumerate(goal.flatten())}
    dist = 0
    for x in range(size):
        for y in range(size):
            val = state[x, y]
            if val != 0:  # Exclude the empty tile
                goal_x, goal_y = goal_positions[val]
                dist += abs(x - goal_x) + abs(y - goal_y)
    return dist


def linear_conflict(state: np.ndarray, goal:np.ndarray) -> int:
    size = state.shape[0]
    conflicts = 0
    for i in range(size):
        # Row conflicts
        row = state[i, :]
        goal_row = goal[i, :]
        for j in range(size):
            if row[j] != 0 and row[j] in goal_row:
                correct_col = np.where(goal_row == row[j])[0][0]
                for k in range(j + 1, size):
                    if row[k] != 0 and row[k] in goal_row:
                        correct_col_k = np.where(goal_row == row[k])[0][0]
                        if correct_col_k < correct_col:
                            conflicts += 2

        # Column conflicts
        col = state[:, i]
        goal_col = goal[:, i]
        for j in range(size):
            if col[j] != 0 and col[j] in goal_col:
                correct_row = np.where(goal_col == col[j])[0][0]
                for k in range(j + 1, size):
                    if col[k] != 0 and col[k] in goal_col:
                        correct_row_k = np.where(goal_col == col[k])[0][0]
                        if correct_row_k < correct_row:
                            conflicts += 2
    return conflicts


def combined_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    return manhattan_distance(state, goal) + linear_conflict(state, goal)


def a_star_search(initial_state: np.ndarray, goal: np.ndarray):
    size = initial_state.shape[0]
    fronteer = []
    heappush(fronteer, (0, initial_state.tobytes(), [], 0))
    visited = set()
    
    while fronteer:
        estimated_cost, current_state_bytes, path, g_cost = heappop(fronteer)
        
        # Convert the bytes back to an array for further processing
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((size, size))

        # Check if we reached the goal
        if is_goal(current_state, goal):
            return path, len(path)

        # Mark the state as visited
        visited.add(current_state_bytes)

        # Explore neighbors
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in visited:
                new_g_cost = g_cost + 0.5
                estimated_cost = new_g_cost + combined_heuristic(new_state, goal)
                heappush(fronteer, (estimated_cost, new_state_bytes, path + [action], new_g_cost))

    return None, 0  # If no solution found


In [100]:
for size in PUZZLE_SIZES:
        print(f"Solving {size}x{size} puzzle...")
        start_time = time.time()
        goal = np.roll(np.arange(size ** 2), -1).reshape((size, size))
        puzzle = generate_random(size)
        print("Initial Puzzle:")
        print(puzzle)
        print("Goal State:")
        print(goal)

        solution = a_star_search(puzzle, goal)
        end_time = time.time()
        print(f"Solved in {solution[1]} moves!")
        print(f"Solved in {end_time - start_time:.2f} seconds")
        print(f"Evaluated {available_actions.calls} actions")
        available_actions.calls = 0
        print("\n--------------------\n")

Solving 3x3 puzzle...
Initial Puzzle:
[[1 7 8]
 [3 0 6]
 [5 2 4]]
Goal State:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Solved in 26 moves!
Solved in 0.31 seconds
Evaluated 284206 actions

--------------------

Solving 4x4 puzzle...
Initial Puzzle:
[[ 4  9 11  3]
 [12  8  1  5]
 [ 0  7 15 10]
 [ 2 13  6 14]]
Goal State:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Solved in 72 moves!
Solved in 0.50 seconds
Evaluated 319959 actions

--------------------

Solving 5x5 puzzle...
Initial Puzzle:
[[24  8  2 15 16]
 [22 18  1  3 10]
 [13 12  0  7 23]
 [21  6 19  9 17]
 [20 14 11  4  5]]
Goal State:
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]]
Solved in 124 moves!
Solved in 62.94 seconds
Evaluated 737413 actions

--------------------

